In [1]:
import pandas as pd
import json

In [2]:
skoly = pd.read_csv("schools_with_villages.csv")[["village", "school", "num_students"]]

In [6]:
skoly

,village,school,num_students
0,Abovce,ZŠ s MŠ - Abovce,34.0
1,Abovce,Materská škola pri ZŠ - Abovce,14.0
2,Abrahám,"Materská škola - Abrahám, Hviezdoslavova",39.0
3,Abrahám,"ZŠ - Abrahám, Školská",130.0
4,Abrahámovce,Materská škola - Abrahámovce,10.0
...,...,...,...
6566,Žlkovce,ZŠ - Žlkovce,29.0
6567,Župčany,ZŠ s MŠ - Župčany,55.0
6568,Župčany,EP ZŠ s MŠ (MŠ) - Župčany,39.0
6569,Župkov,Materská škola pri ZŠ - Župkov,25.0


In [75]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# obce = pd.read_csv("https://raw.githubusercontent.com/cedeerwe/korona-data-uderka/master/pairs.csv", usecols=[1,3], names=["village", "district"])

In [76]:
# obce_map = {obec["village"]: obec["district"] for i, obec in obce.iterrows()}
# obce_map["Senica"]

In [77]:
len(obce)

4208

In [113]:
with open("../obyvatelia/obce_obyvatelia_fixed.json", "r") as f:
    obyvatelia = json.loads(f.read())
obyvatelia = pd.DataFrame([{"village": obec["obec"], "population": obec["pocet_obyvatelov_2013"], "district": obec['okres']} for obec in obyvatelia])
obyvatelia['district'] = obyvatelia['district'].apply(lambda val: str(val).format('utf8'))
obyvatelia['village'] = obyvatelia['village'].apply(lambda val: str(val).format('utf8'))


"Å enkvice".format('utf-8')


'Å\xa0enkvice'

In [91]:
len(obyvatelia)

2927

In [92]:
obyvatelia_with_district = obyvatelia

In [127]:
days = 3
per_day = 500
population_split_threshold = days * per_day
filtered_obce = obyvatelia_with_district[obyvatelia_with_district['population'] >= population_split_threshold]


In [128]:
obyvatelia_per_district = {
    district: obyvatelia_with_district[
        obyvatelia_with_district['district'] == district
    ]['population'].sum() 
    for district in obyvatelia_with_district['district'].unique()}

In [133]:
obce_so_skolami = set(skoly['village'])
tests_per_day_per_odberove_miesto = 500 # 0700 - 1900

def sort_dict_by_keys(d):
    return dict(sorted(d.items(), key=lambda x: x[0].lower()))

population_split_threshold_const = 3
def skola_v_obci(obec):
    if "Å¡" in obec['village']:
        obec['village'] = obec['village'].replace("Å¡", "š")
        obec['village'] = obec['village'].replace("Å¡", "š")
    if "Å " in obec['village']:
        obec['village'] = obec['village'].replace('Å ', "Š")
        obec['village'] = obec['village'].replace('Å ', "Š")
        
    if obec['population'] > population_split_threshold_const * population_split_threshold:
        return "Nechávame na uvážení obce"
            
    if obec['village'] == "Šaštín - Stráže":
        return "Základná umelecká škola - Šaštín-Stráže, J.Hollého"
    elif obec['village'] == "Cabaj - Čápor":
        return "ZŠ - Cabaj-Čápor, časť Cabaj"
    elif obec['village'] in obce_so_skolami:
        obec_df = skoly[skoly["village"] == obec['village']]
        skoly_obec_studenti = obec_df[obec_df['num_students'].notnull()]
        if len(skoly_obec_studenti) == 0:
            return obec_df["school"].values[0]
        else:
            return skoly_obec_studenti.iloc[skoly_obec_studenti['num_students'].argmax()]['school']
    else:
        if ("Bratislava" in obec['village'] or "Košice" in obec['village']):
            return "Nechávame na uvážení okresu"
        else:
            print("Chyba, nemame skolu pre: ", obec['village'])
            return "Nechávame na uvážení obce"

from collections import defaultdict      
result = defaultdict(list)

for i, obec in filtered_obce.iterrows():
    district = obec["district"]
    
    result_for_obec = {
        "Obec": obec["village"],
        "Navrhované odberové miesto": skola_v_obci(obec),
        "Počet obyvateľov": obec["population"],
        # "Navrhovaný počet odberových jednotiek": "{:.2f}".format(obec["population"] / 5000)
    }
    result[district].append(result_for_obec)

result2 = {}
for district, district_obce in result.items():
    pocet_odberovych_miest = obyvatelia_per_district[district] / population_split_threshold
    real_pocet_odberovych_miest = 0
    obyvatelia_v_odberovych = sum(obec['Počet obyvateľov'] for obec in district_obce)
    odberove = []
    for obec in district_obce:
        odberovych = round(obec["Počet obyvateľov"] / obyvatelia_v_odberovych * pocet_odberovych_miest)
        real_pocet_odberovych_miest += odberovych
        tests_per_day = odberovych * tests_per_day_per_odberove_miesto
        obec["Navrhovaný počet odberových jednotiek pre obec"] = odberovych
        obec["Kapacita testov za deň pre obec"] = tests_per_day
        obec["Kapacita testov za týždeň pre obec"] = 7 * tests_per_day
        odberove.append(obec)
    odberove_sorted = sorted(odberove, key=lambda odberove: str(odberove["Obec"]), reverse=False)
    result2[district] = {
        "Navrhovaný počet odberových miest pre okres": real_pocet_odberovych_miest,
        "Odberové miesta": odberove_sorted,
    }


# to csv
csv_rows = []
sorted_result = sort_dict_by_keys(result2)

for district, value in sorted_result.items():
    tests_per_day = value['Navrhovaný počet odberových miest pre okres'] * tests_per_day_per_odberove_miesto
    csv_rows.append({
        "Okres": district,
        "Počet obyvateľov v okrese": obyvatelia_per_district[district],
        "Navrhovaný počet odberových miest pre okres": value["Navrhovaný počet odberových miest pre okres"],
        "Kapacita testov za deň pre okres": tests_per_day,
        "Kapacita testov za týždeň pre okres": tests_per_day * 7
    })
    
    for miesto in value['Odberové miesta']:
        csv_rows.append(miesto)


# sorted_result
pd.DataFrame(csv_rows).to_csv("odberove_miesta_{}_{}.csv".format(population_split_threshold, tests_per_day_per_odberove_miesto))       

Chyba, nemame skolu pre:  Plevník - Drienové
Chyba, nemame skolu pre:  Nedožery - Brezany
Chyba, nemame skolu pre:  Chocholná - Velčice
Chyba, nemame skolu pre:  Melčice - Lieskové
Chyba, nemame skolu pre:  Výčapy - Opatovce
Chyba, nemame skolu pre:  Sliače
Chyba, nemame skolu pre:  Lietavská Svinná - Babkov
Chyba, nemame skolu pre:  Hodruša - Hámre
Chyba, nemame skolu pre:  Lovčica - Trubín


In [122]:
import yaml
with open("odberove_miesta.txt", "w") as f:
    yaml.dump(result2, f)

In [78]:
set(filtered_obce['village']) - set(result.keys())

set()

In [81]:
len(set(filtered_obce['village']))


144